# Improve the Microscope Pictures with a Camera, a NeoPixel Illuminator and Fourier Ptychography

by [Jose David Marroquin Toledo](https://github.com/josemarroquintoledo/)

## 1 Import Functions from Other Jupyter Notebooks

In [ ]:
%run ../jupyter-modules/fwdimaging.ipynb
%run ../jupyter-modules/phaseretrieval.ipynb
%run ../jupyter-modules/microscopx.ipynb
%run ../jupyter-modules/fputilities.ipynb

## 2 Takes the Photos

In [ ]:
N_PHOTOS = 47
CAM = 0  # Run ls -ltrh /dev/video* to list the video device picked up
         # by the Linux kernel.
project_path = take_photos(CAM, N_PHOTOS)

## 3 Generate the Incident Wave Vectors for the NeoPixel Lamp

In [ ]:
import matplotlib.pyplot as plt

SAMPLE_DISTANCE = 47  # Distance between in mm between the NeoPixel
                      # illuminator and the sample.
# {<Pixels per ring>: <angle respect to the horizontal in degrees>,
#  ... }
PIXEL_LAMP = {7: 0, 16: -11.25, 24: -7.5}

xy_lst_per_ring = get_pixels_xy(rings=PIXEL_LAMP)
xy_lst_lst= sorted(xy_lst_per_ring.values())
xy_lst = list()
for lst in xy_lst_lst:
    # Concatenate all list of xy_lst_lst.
    xy_lst.extend(lst)
wv_xy = gen_wave_vectors(xy_lst, SAMPLE_DISTANCE)

## 4 Import the Lo-Res Image Set

In [ ]:
lores_set = read_lores_img_set(project_path, N_PHOTOS, format='PNG')
hires_w_px = lores_set.shape[2] * 4  # Width of the reconstruction in px.
hires_h_px = lores_set.shape[1] * 4  # Height of the reconstruction in
                                     # px.

## 5 Set a Reconstruction Sequence

In [ ]:
# Generate a sequence that starts in the center of the Fourier spectrum
# (central Pixel).
seq = list(range(1, N_PHOTOS + 1))

## 6 Reconstruct the Hi-Res Image from the Lo-Res Image Set

In [ ]:
WAVELENGTH = 0.63e-6  # In m.
CCD_PX = 2.75e-6  # The size of the employed CCD in m.
NA = 0.08  # Numerical aperture (NA) of the objective lens.

# Generate the coherent transfer function of the coherent imaging
# system.
cft, lores_w_px, lores_h_px, dkx, dky, kx, ky = gen_cft(np.array(wv_xy),
    WAVELENGTH, CCD_PX, NA, hires_w_px, hires_h_px)

# Reconstruct the Hi-Res image.
gen_hires_img(lores_set, seq, hires_w_px, hires_h_px, cft, lores_w_px,
              lores_h_px, dkx, dky, kx, ky, project_path)